In [ ]:
# in this notebook we see how to realign images
# this is often needed as telescope guidance is never perfect
# and a lot can occur that will displace the pointing:
# - some clouds come, guidance is lost, then reacquired
# - if we want to combine images taken from 2 different nights or more

# make figures interactive:
#%matplotlib notebook 
%matplotlib inline
# import library containing image reduction tools for this class
from image_reduction_tools import *
# packages for realignment:
import astroalign as aa
import numpy as np
from astropy.io import fits
import astropy.io.fits as io


In [ ]:
# first lets look at the problem: we would like to RGB compose an image of M1:
# we want the longest exposures in Halpha, OIII and B:
files=glob.glob('../../data/photo/M1/p*f1.fits')
i=0
for file in files:
    print(i)
    print(file)
    print_header(file)
    i=i+1

In [ ]:
# so our files of interest (for M1) are 0, 3, and 5:
B_file=files[0]
OIII_file=files[3]
Ha_file=files[5]

# we could work with that but for clarity we will work in data/photo/M1/align
# and create 3 new files there (using cp in the terminal):
#B.fits    Ha.fits   OIII.fits

B_file="../../data/photo/M1/align/B.fits"
OIII_file="../../data/photo/M1/align/OIII.fits"
Ha_file="../../data/photo/M1/align/Ha.fits"


# we can easily see they are misaligned:
B=fits.getdata(B_file,ext=0)
OIII=fits.getdata(OIII_file,ext=0)

plt.figure(figsize=(5,5))
plt.imshow(np.log10(B[:150,:150]))
plt.show()
plt.figure(figsize=(5,5))
plt.imshow(np.log10(OIII[:150,:150]))
plt.show()



In [ ]:
# generate the new filenames for the aligned images:
import os
# Split the filename into base and extension
base_OIII, extension = os.path.splitext(OIII_file)
base_Ha, extension = os.path.splitext(Ha_file)

# insert_align before the .fits
OIII_al_file=f"{base_OIII}_align{extension}"
Ha_al_file=f"{base_Ha}_align{extension}"
print(OIII_al_file)
print(Ha_al_file)


In [ ]:
# now, lets do the 'registering', i.e. the action of 
# detecting stars and using them to find the offset between the images:
# align OIII trying to match B. OIII_al is the result.
OIII_al,footprint=aa.register(np.float32(OIII),np.float32(B))
# now lets look at OIII_al and B
plt.figure(figsize=(5,5))
plt.imshow(np.log10(B[:150,:150]))
plt.show()
plt.figure(figsize=(5,5))
plt.imshow(np.log10(OIII_al[:150,:150]))
plt.show()
# write the new aligned image
write_fits_image(OIII_al_file,OIII_al)

# lets do the same with the Halpha image just to be sure
Ha=fits.getdata(Ha_file,ext=0)
Ha_al,footprint=aa.register(np.float32(Ha),np.float32(B))
write_fits_image(Ha_al_file,Ha_al)


In [ ]:
# Nice, now its easy to compose a RGB image with ds9, outside of python